In [385]:
import cv2
import dlib
import numpy as np

# Khởi tạo Dlib
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


In [386]:
# Các biến toàn cục
current_effect = None  # Hiệu ứng hiện tại
left_effects = ["Negative", "Sepia", "Wide Angle View", "Edge", "Black & White", "Cool Tone","Brighten & Contrast","Remove Blemishes", "Reset"]
right_effects = ["Glasses", "Dog Nose", "Xmas Hat", "Frog Hat", "Bunny Hat", "Cat 1", "Cat 2","Cat w Glasses","Flower"]

In [387]:
import math

def apply_negative(image):
    return 255 - image

def apply_canny_edge(frame, low_threshold=50, high_threshold=150):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 1.5)  # Làm mờ để giảm nhiễu
    edges = cv2.Canny(blurred, low_threshold, high_threshold, apertureSize=3, L2gradient=True)
    return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)


def apply_sepia(image):
    sepia_filter = np.array([[0.272, 0.534, 0.131],
                             [0.349, 0.686, 0.168],
                             [0.393, 0.769, 0.189]], dtype=np.float32)
    image = np.array(image, dtype=np.float32) / 255.0
    sepia_image = cv2.transform(image, sepia_filter)
    sepia_image = np.clip(sepia_image * 255, 0, 255).astype(np.uint8)
    return sepia_image

def apply_wide(image):
    h, w = image.shape[:2]
    K = np.array([[w, 0, w // 2], [0, h, h // 2], [0, 0, 1]], dtype=np.float32)
    D = np.array([0.7, -0.5, 0.0, 0.0], dtype=np.float32)  # Tăng độ méo để hiệu ứng rõ hơn
    map1, map2 = cv2.initUndistortRectifyMap(K, D, np.eye(3), K, (w, h), cv2.CV_32FC1)
    return cv2.remap(image, map1, map2, interpolation=cv2.INTER_LINEAR)

def apply_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def apply_cool(image):
    decrease = np.array([0, 10, 20], dtype=np.uint8)
    cool_image = cv2.subtract(image, decrease)
    return np.clip(cool_image, 0, 255)

def apply_histogram_equalization(image):
# Chuyển ảnh sang không gian màu YCrCb
    ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    
    # Tách các kênh màu
    y, cr, cb = cv2.split(ycrcb)
    
    # Cân bằng histogram trên kênh sáng (Y channel)
    y_equalized = cv2.equalizeHist(y)
    
    # Gộp lại các kênh
    ycrcb_equalized = cv2.merge((y_equalized, cr, cb))
    
    # Chuyển về không gian màu BGR
    image = cv2.cvtColor(ycrcb_equalized, cv2.COLOR_YCrCb2BGR)
    return image

def calculate_face_angle(landmarks):
    # Lấy vị trí hai mắt
    left_eye = (landmarks.part(36).x, landmarks.part(36).y)
    right_eye = (landmarks.part(45).x, landmarks.part(45).y)

    # Tính toán góc nghiêng khuôn mặt
    delta_x = right_eye[0] - left_eye[0]
    delta_y = right_eye[1] - left_eye[1]
    angle = math.degrees(math.atan2(delta_y, delta_x))  # Tính góc (đơn vị độ)

    # Đảo dấu góc để phù hợp với hướng nghiêng
    return -angle

def rotate_filter(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)

    # Tính ma trận xoay
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    cos = abs(rotation_matrix[0, 0])
    sin = abs(rotation_matrix[0, 1])

    # Tính kích thước khung hình mới
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))

    # Điều chỉnh ma trận xoay
    rotation_matrix[0, 2] += (new_w / 2) - center[0]
    rotation_matrix[1, 2] += (new_h / 2) - center[1]

    # Xoay ảnh
    return cv2.warpAffine(image, rotation_matrix, (new_w, new_h), borderValue=(0, 0, 0, 0))



In [388]:
def add_glasses(image, landmarks):
    glasses = cv2.imread("glasses.png", -1)  # PNG image with transparency
    if glasses is None:
        print("Glasses image not found!")
        return image

    # Tăng tỉ lệ kính bằng scale_factor
    scale_factor = 1.5  # Điều chỉnh giá trị này để làm kính lớn hơn hoặc nhỏ hơn
    glasses_width = int((landmarks.part(45).x - landmarks.part(36).x) * scale_factor)
    glasses_height = int(glasses_width * glasses.shape[0] / glasses.shape[1])
    angle = calculate_face_angle(landmarks)
    glasses = rotate_filter(glasses, angle)
    # Resize kính theo chiều rộng và chiều cao tính toán
    glasses = cv2.resize(glasses, (glasses_width, glasses_height))

    # Tính tọa độ trên-trái kính dựa trên mắt trái (landmark 36)
    top_left = (landmarks.part(36).x - int((glasses_width - (landmarks.part(45).x - landmarks.part(36).x)) / 2),
                landmarks.part(36).y - glasses_height // 2)

    # Kiểm tra nếu vùng kính nằm ngoài ảnh để tránh lỗi
    if (top_left[1] + glasses_height > image.shape[0] or
            top_left[0] + glasses_width > image.shape[1] or
            top_left[0] < 0 or top_left[1] < 0):
        print("Glasses exceed image boundaries")
        return image

    # Gắn kính vào khuôn mặt bằng alpha blending
    for c in range(0, 3):
        roi = image[top_left[1]:top_left[1] + glasses_height, top_left[0]:top_left[0] + glasses_width, c]
        alpha = glasses[:, :, 3] / 255.0
        image[top_left[1]:top_left[1] + glasses_height, top_left[0]:top_left[0] + glasses_width, c] = (
        glasses[:, :, c] * alpha + roi * (1 - alpha)
    )


    return image

In [389]:
def add_dog_nose(image, landmarks):
    nose_image = cv2.imread("dog_nose.png", cv2.IMREAD_UNCHANGED)
    nose_width = (landmarks.part(35).x - landmarks.part(31).x) * 5
    angle = calculate_face_angle(landmarks)
    nose_image = rotate_filter(nose_image, angle)
    nose_image = cv2.resize(nose_image, (nose_width, nose_width))

    x = landmarks.part(30).x - nose_width // 2
    y = landmarks.part(30).y - nose_width // 2
    for i in range(nose_image.shape[0]):
        for j in range(nose_image.shape[1]):
            if nose_image[i, j, 3] > 0:
                image[y + i, x + j] = nose_image[i, j, :3]
    
    return image

In [390]:
def add_hat(image, landmarks, scale=1.5, height_offset=0):
    # Đọc ảnh mũ
    hat_image = cv2.imread("xmas_hat.png", cv2.IMREAD_UNCHANGED)

    angle = calculate_face_angle(landmarks)
    hat_image = rotate_filter(hat_image, angle)
    
    # Xác định kích thước và vị trí đặt mũ
    head_width = (landmarks.part(16).x - landmarks.part(0).x) * scale
    hat_width = int(head_width)
    hat_height = int(hat_width * hat_image.shape[0] / hat_image.shape[1])
    hat_image = cv2.resize(hat_image, (hat_width, hat_height))

    # Tính toán vị trí đặt mũ (dựa trên landmarks phần trán)
    x = landmarks.part(27).x - hat_width // 2 + 60  # Căn giữa theo điểm 27 (đỉnh mũi)
    y = landmarks.part(19).y - hat_height + height_offset  # Đặt trên trán, dịch lên nếu cần

    # Đảm bảo vị trí không vượt ngoài khung hình
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(image.shape[1], x1 + hat_width)
    y2 = min(image.shape[0], y1 + hat_height)

    # Điều chỉnh vùng hiển thị của filter nếu bị cắt
    hat_image = hat_image[:y2 - y1, :x2 - x1]

    # Áp dụng mũ với alpha blending
    for c in range(0, 3):  # Lặp qua các kênh màu (B, G, R)
        alpha = hat_image[:, :, 3] / 255.0  # Kênh alpha của mũ
        image[y1:y2, x1:x2, c] = (hat_image[:, :, c] * alpha +
                                  image[y1:y2, x1:x2, c] * (1 - alpha))
    return image


In [391]:
def add_frog_hat(image, landmarks, scale=3.2, height_offset=-270):
    # Đọc ảnh mũ
    hat_image = cv2.imread("frog_hat.png", cv2.IMREAD_UNCHANGED)

    # Xác định các điểm chính
    left_ear = (landmarks.part(0).x, landmarks.part(0).y)  # Tai trái
    right_ear = (landmarks.part(16).x, landmarks.part(16).y)  # Tai phải
    top_head = (landmarks.part(27).x, landmarks.part(27).y)  # Trán (đỉnh mũi)

    # Tính toán chiều rộng và chiều cao mũ
    face_width = int(math.sqrt((right_ear[0] - left_ear[0]) ** 2 + (right_ear[1] - left_ear[1]) ** 2) * scale)
    face_height = int((landmarks.part(8).y - top_head[1]) * scale)

    # Resize mũ để khớp với chiều rộng khuôn mặt
    hat_width = face_width
    hat_height = int(hat_width * hat_image.shape[0] / hat_image.shape[1])
    hat_image = cv2.resize(hat_image, (hat_width, hat_height))

    # Tính góc nghiêng ngang của khuôn mặt
    delta_x = right_ear[0] - left_ear[0]
    delta_y = right_ear[1] - left_ear[1]
    angle = math.degrees(math.atan2(delta_y, delta_x))

    # Xoay mũ theo góc nghiêng ngang
    center = (hat_width // 2, hat_height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)
    hat_image = cv2.warpAffine(hat_image, rotation_matrix, (hat_width, hat_height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # Tính toán vị trí đặt mũ
    x = top_head[0] - hat_width // 2
    y = top_head[1] - hat_height + height_offset

    # Đảm bảo không vượt ngoài khung hình
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(image.shape[1], x1 + hat_width)
    y2 = min(image.shape[0], y1 + hat_height)

    # Điều chỉnh kích thước mũ nếu bị cắt
    hat_image = hat_image[:y2 - y1, :x2 - x1]

    # Áp dụng mũ với alpha blending
    for c in range(0, 3):  # Lặp qua các kênh màu (B, G, R)
        alpha = hat_image[:, :, 3] / 255.0  # Kênh alpha của mũ
        image[y1:y2, x1:x2, c] = (hat_image[:, :, c] * alpha +
                                  image[y1:y2, x1:x2, c] * (1 - alpha))
    return image


In [392]:
def add_bunny_hat(image, landmarks, height_offset=350):
    # Đọc ảnh mũ
    hat_image = cv2.imread("bunny_hat.png", cv2.IMREAD_UNCHANGED)
    if hat_image is None:
        print("Error: Bunny hat image not found!")
        return image

    # Xác định các điểm chính
    try:
        nose = (landmarks.part(30).x, landmarks.part(30).y)  # Vị trí mũi
        left_ear = (landmarks.part(0).x, landmarks.part(0).y)  # Tai trái
        right_ear = (landmarks.part(16).x, landmarks.part(16).y)  # Tai phải
        top_head = (landmarks.part(27).x, landmarks.part(27).y)  # Đỉnh đầu
    except AttributeError:
        print("Error: Invalid landmarks detected!")
        return image

    # Tính chiều rộng và chiều cao khuôn mặt
    face_width = math.sqrt((right_ear[0] - left_ear[0]) ** 2 + (right_ear[1] - left_ear[1]) ** 2)
    face_height = top_head[1] - nose[1]

    # Tính kích thước mũ
    hat_width = int(face_width * 3.2)
    hat_height = int(hat_width * hat_image.shape[0] / hat_image.shape[1])
    hat_image = cv2.resize(hat_image, (hat_width, hat_height), interpolation=cv2.INTER_CUBIC)

    # Tính góc nghiêng khuôn mặt
    delta_x = right_ear[0] - left_ear[0]
    delta_y = right_ear[1] - left_ear[1]
    angle = math.degrees(math.atan2(delta_y, delta_x))

    # Xoay mũ theo góc nghiêng
    center = (hat_width // 2, hat_height // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, -angle, 1.0)
    rotated_hat = cv2.warpAffine(hat_image, rotation_matrix, (hat_width, hat_height), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))

    # Đặt tâm khoảng trống trong mũ trùng với mũi
    x = int(nose[0] - hat_width // 2)
    y = int(top_head[1] - hat_height // 1.5) - height_offset

    # Đảm bảo không vượt ngoài khung hình
    x1 = max(0, x)
    y1 = max(0, y)
    x2 = min(image.shape[1], x1 + hat_width)
    y2 = min(image.shape[0], y1 + hat_height)

    # Cắt mũ nếu vượt khung hình
    rotated_hat = rotated_hat[:y2 - y1, :x2 - x1]

    # Áp dụng mũ với alpha blending
    for c in range(0, 3):
        alpha = rotated_hat[:, :, 3] / 255.0  # Kênh alpha
        if alpha.shape[0] > 0 and alpha.shape[1] > 0:
            image[y1:y2, x1:x2, c] = (rotated_hat[:, :, c] * alpha +
                                      image[y1:y2, x1:x2, c] * (1 - alpha))
    return image


In [393]:
def add_cat_face(image, landmarks):
    # Đọc ảnh filter
    cat_filter = cv2.imread("meo.png", cv2.IMREAD_UNCHANGED)
    angle = calculate_face_angle(landmarks)
    cat_filter = rotate_filter(cat_filter, angle)
    
    # Xác định kích thước khuôn mặt
    face_width = landmarks.part(16).x - landmarks.part(0).x
    face_height = landmarks.part(8).y - landmarks.part(19).y

    # Resize filter để khớp với khuôn mặt
    filter_width = int(face_width * 1.4)  # Tăng kích thước filter theo chiều ngang
    filter_height = int(filter_width * cat_filter.shape[0] / cat_filter.shape[1] * 1.3)  # Tăng kích thước filter theo chiều dọc
    cat_filter = cv2.resize(cat_filter, (filter_width, filter_height))

    # Tính toán vị trí filter
    x = landmarks.part(27).x - filter_width // 2  # Đặt trung tâm filter tại đỉnh mũi
    y = landmarks.part(19).y - filter_height // 3  # Đặt filter phía trên trán

    # Overlay filter lên ảnh gốc
    for i in range(cat_filter.shape[0]):
        for j in range(cat_filter.shape[1]):
            if y + i >= image.shape[0] or x + j >= image.shape[1]:
                continue
            if cat_filter[i, j, 3] > 0:  # Kênh alpha > 0
                image[y + i, x + j] = cat_filter[i, j, :3]  # Chỉ lấy RGB từ filter

    return image


In [394]:
def add_cat2_face(image, landmarks):
    # Đọc ảnh filter
    cat_filter = cv2.imread("meomeo.png", cv2.IMREAD_UNCHANGED)
    angle = calculate_face_angle(landmarks)
    cat_filter = rotate_filter(cat_filter, angle)
    
    # Xác định kích thước khuôn mặt
    face_width = landmarks.part(16).x - landmarks.part(0).x
    face_height = landmarks.part(8).y - landmarks.part(19).y

    # Resize filter để khớp với khuôn mặt
    filter_width = int(face_width * 1.2)  # Tăng kích thước filter theo chiều ngang
    filter_height = int(filter_width * cat_filter.shape[0] / cat_filter.shape[1])  # Tăng kích thước filter theo chiều dọc
    cat_filter = cv2.resize(cat_filter, (filter_width, filter_height))

    # Tính toán vị trí filter
    x = landmarks.part(27).x - filter_width // 2  # Đặt trung tâm filter tại đỉnh mũi
    y = landmarks.part(19).y - filter_height // 3 - 50 # Đặt filter phía trên trán

    # Overlay filter lên ảnh gốc
    for i in range(cat_filter.shape[0]):
        for j in range(cat_filter.shape[1]):
            if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:
                if cat_filter[i, j, 3] > 0:  # Kênh alpha > 0
                 image[y + i, x + j] = cat_filter[i, j, :3]  # Chỉ lấy RGB từ filter
    return image


In [395]:
def add_cat3_face(image, landmarks):
    # Đọc ảnh filter
    cat = cv2.imread("cute_cat.png", cv2.IMREAD_UNCHANGED)
    if cat is None:
        print("Error: Filter image not found!")
        return image

    # Tính góc nghiêng khuôn mặt
    angle = calculate_face_angle(landmarks)
    cat = rotate_filter(cat, angle)

    # Xác định kích thước khuôn mặt
    face_width = landmarks.part(16).x - landmarks.part(0).x
    face_height = landmarks.part(8).y - landmarks.part(19).y

    # Tính toán kích thước filter
    filter_width = int(face_width * 1.2)  # Tăng kích thước filter theo chiều ngang
    filter_height = int(filter_width * cat.shape[0] / cat.shape[1])  # Tăng kích thước filter theo chiều dọc

    # Đảm bảo kích thước filter không vượt quá khung hình
    filter_width = min(filter_width, image.shape[1])
    filter_height = min(filter_height, image.shape[0])

    # Resize filter
    cat = cv2.resize(cat, (filter_width, filter_height), interpolation=cv2.INTER_CUBIC)

    # Tính toán vị trí filter
    x = landmarks.part(27).x - filter_width // 2  # Đặt trung tâm filter tại đỉnh mũi
    y = landmarks.part(19).y - filter_height // 3 - 55 # Đặt filter phía trên trán

    # Overlay filter lên ảnh gốc
    for i in range(cat.shape[0]):
        for j in range(cat.shape[1]):
            if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:  # Giới hạn tọa độ
                alpha = cat[i, j, 3] / 255.0  # Lấy kênh alpha
                if alpha > 0:  # Chỉ xử lý pixel có alpha > 0
                    image[y + i, x + j] = (alpha * cat[i, j, :3] + (1 - alpha) * image[y + i, x + j]).astype(np.uint8)

    return image


In [396]:
def add_flower_crown(image, landmarks):
    # Đọc ảnh filter bông hoa
    flower_crown = cv2.imread("flower.png", cv2.IMREAD_UNCHANGED)
    if flower_crown is None:
        print("Error: Flower image not found!")
        return image

    # Xác định kích thước khuôn mặt
    face_width = landmarks.part(16).x - landmarks.part(0).x  # Từ tai trái đến tai phải
    crown_width = int(face_width * 1.5)  # Tăng kích thước để phủ toàn bộ đầu
    crown_height = int(crown_width * flower_crown.shape[0] / flower_crown.shape[1])  # Duy trì tỉ lệ ảnh
    flower_crown = cv2.resize(flower_crown, (crown_width, crown_height))

    # Tính toán vị trí để đặt bông hoa (trên trán)
    x = landmarks.part(27).x - crown_width // 2  # Đặt trung tâm tại đỉnh mũi
    y = landmarks.part(19).y - crown_height // 2 - 50  # Đặt filter phía trên trán

    # Dán filter lên ảnh gốc
    for i in range(flower_crown.shape[0]):
        for j in range(flower_crown.shape[1]):
            if 0 <= y + i < image.shape[0] and 0 <= x + j < image.shape[1]:  # Giới hạn tọa độ
                alpha = flower_crown[i, j, 3] / 255.0  # Kênh alpha
                if alpha > 0:  # Chỉ dán những pixel có alpha > 0
                    image[y + i, x + j] = alpha * flower_crown[i, j, :3] + (1 - alpha) * image[y + i, x + j]

    return image


In [397]:
def remove_blemishes_inpaint(image, threshold=200, inpaint_radius=3):
    """
    Xóa mụn bằng kỹ thuật Inpainting (phục hồi ảnh).
    
    Tham số:
    - image: Ảnh đầu vào (BGR).
    - threshold: Giá trị ngưỡng để phát hiện mụn (0-255).
    - inpaint_radius: Bán kính vùng phục hồi (inpainting).

    Trả về:
    - result: Ảnh đã xử lý.
    """
    # Chuyển ảnh sang màu xám để phát hiện vùng mụn
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Tạo mask cho các vùng mụn dựa trên ngưỡng
    _, mask = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # Áp dụng inpainting để phục hồi vùng được phát hiện
    result = cv2.inpaint(image, mask, inpaint_radius, cv2.INPAINT_TELEA)

    return result


In [398]:
def on_mouse(event, x, y, flags, param):
    global current_effect
    if event == cv2.EVENT_LBUTTONDOWN:
        frame_width = param["frame_width"]
        frame_height = param["frame_height"]

        button_width = 150
        button_height = 40
        padding = 10

        # Kiểm tra nút bên trái
        max_buttons_per_column = (frame_height - padding) // (button_height + padding)
        for i, effect in enumerate(left_effects):
            x1 = padding
            y1 = padding + i * (button_height + padding)
            x2 = x1 + button_width
            y2 = y1 + button_height
            if x1 <= x <= x2 and y1 <= y <= y2:
                current_effect = effect
                print(f"Selected Effect: {effect}")
                return

        # Kiểm tra nút bên phải
        for i, effect in enumerate(right_effects):
            x1 = frame_width - button_width - padding
            y1 = padding + i * (button_height + padding)
            x2 = x1 + button_width
            y2 = y1 + button_height
            if x1 <= x <= x2 and y1 <= y <= y2:
                current_effect = effect
                print(f"Selected Effect: {effect}")
                return

def draw_buttons(frame):
    """
    Vẽ các nút bấm bên trái và phải màn hình.
    """
    button_width = 150
    button_height = 40
    padding = 10

    # Vẽ nút bên trái
    for i, effect in enumerate(left_effects):
        x1 = padding
        y1 = padding + i * (button_height + padding)
        x2 = x1 + button_width + 55
        y2 = y1 + button_height
        color = (200, 200, 200) if effect != current_effect else (100, 200, 100)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, -1)
        cv2.putText(frame, effect, (x1 + 10, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

    # Vẽ nút bên phải
    frame_width = frame.shape[1]
    for i, effect in enumerate(right_effects):
        x1 = frame_width - button_width - padding
        y1 = padding + i * (button_height + padding)
        x2 = x1 + button_width
        y2 = y1 + button_height
        color = (200, 200, 200) if effect != current_effect else (100, 200, 100)
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, -1)
        cv2.putText(frame, effect, (x1 + 10, y1 + 25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)


In [399]:
# Main loop
cap = cv2.VideoCapture(0)
cv2.namedWindow("Selfie Beautification Filter")
cv2.setMouseCallback("Selfie Beautification Filter", on_mouse)

ret, frame = cap.read()
frame_height, frame_width = frame.shape[:2]
cv2.setMouseCallback("Selfie Beautification Filter", on_mouse, {
    "frame_width": frame_width,
    "frame_height": frame_height
})

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    original_frame = frame.copy()  # Giữ lại bản gốc
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    landmarks = None 

    if current_effect in right_effects:
        faces = face_detector(gray_frame)
        if len(faces) > 0 :
         landmarks = landmark_predictor(gray_frame, faces[0])
         if current_effect == "Glasses":
           frame = add_glasses(frame, landmarks)
         elif current_effect == "Dog Nose":
           frame = add_dog_nose(frame, landmarks)
         elif current_effect == "Xmas Hat":
           frame = add_hat(frame, landmarks, scale=2)
         elif current_effect == "Frog Hat":
           frame = add_frog_hat(frame, landmarks, scale=3.2,height_offset=-270)
         elif current_effect == "Bunny Hat":
           frame = add_bunny_hat(frame, landmarks, height_offset= 350)
         elif current_effect == "Cat 1":
           frame = add_cat_face(frame, landmarks)
         elif current_effect == "Cat 2":
           frame = add_cat2_face(frame, landmarks)
         elif current_effect == "Cat w Glasses":
           frame = add_cat3_face(frame, landmarks)
         elif current_effect == "Flower":
           frame = add_flower_crown(frame, landmarks)
           
    else:
        if current_effect == "Negative":
            frame = apply_negative(frame)
        elif current_effect == "Sepia":
            frame = apply_sepia(frame)
        elif current_effect == "Wide Angle View":
            frame = apply_wide(frame)
        elif current_effect == "Edge":
            frame = apply_canny_edge(frame, low_threshold=10, high_threshold=90)
        elif current_effect == "Black & White":
            frame = cv2.cvtColor(apply_gray(frame), cv2.COLOR_GRAY2BGR)
        elif current_effect == "Cool Tone":
            frame = apply_cool(frame)
        elif current_effect == "Brighten & Contrast":
            frame = apply_histogram_equalization(frame)  
        elif current_effect == "Remove Blemishes":
          frame = remove_blemishes_inpaint(frame, threshold=200, inpaint_radius=3)     
        elif current_effect == "Reset":
            frame = original_frame
            
    draw_buttons(frame)

    # Hiển thị kết quả
    cv2.imshow("Selfie Beautification Filter", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Selected Effect: Dog Nose
Selected Effect: Xmas Hat
